In [2]:
!pip install langchain_community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [3]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.75
    Uninstalling langchain-core-0.3.75:
      Successfully uninstalled langchain-core-0.3.75


In [1]:
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 16.4 MB/s eta 0:00:00


In [5]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=a7de0df0069da72b52487a5d4293e1ac3d1490cedec36bc7feb40d62cb767571
  Stored in directory: /root/.cache/pip/wheels/0c/47/f5/89b7e770ab2996baf8c910e7353d6391e373075a0ac213519e
Successfully built google-search-results


In [2]:
import os
import operator
from typing import TypedDict, Annotated, List
from dotenv import load_dotenv

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SerpAPIWrapper
from langgraph.graph import StateGraph, END

In [31]:
os.environ["SERPAPI_API_KEY"] = "28c7641827dc44957197ef674a2945932ec69475db9f5b7af4ab08fe9d6fbff5"

llm = ChatOpenAI(
    api_key="KEY FROM OPENROUTER",
    base_url="https://openrouter.ai/api/v1",
    model="deepseek/deepseek-chat-v3.1:free",
    temperature=0.7,
    streaming=True
)

In [32]:
# --- Tool Definition ---
# Instantiate the SerpAPI search tool.
search_tool = SerpAPIWrapper()

# --- State Definition ---
# This defines the "memory" or "state" that flows through the graph.
class ResearchState(TypedDict):
    topic: str
    explanation: str
    summary: str


In [33]:
def researcher_agent(state: ResearchState) -> dict:
    """
    This agent uses a web search tool to find information on a topic and then explains it.
    """
    print("---RESEARCHER (with SerpApi Web Search)---")
    topic = state["topic"]

    # Use the tool to search for information. The .run() method takes the query string.
    search_results = search_tool.run(topic)

    print(f"Search Results:\n{search_results}")

    prompt = ChatPromptTemplate.from_template(
        """You are a helpful research assistant. Based on the following search results,
        provide a brief, easy-to-understand explanation of the topic: {topic}.

        Search Results:
        {search_results}
        """
    )

    chain = prompt | llm
    result = chain.invoke({"topic": topic, "search_results": search_results})

    print(f"Researcher's Explanation:\n{result.content}")
    return {"explanation": result.content}

In [34]:
def summarizer_agent(state: ResearchState) -> dict:
    """
    This agent takes an explanation and summarizes it in one sentence.
    """
    print("---SUMMARIZER---")
    explanation = state["explanation"]

    prompt = ChatPromptTemplate.from_template(
        "You are a summarization expert. Condense the following text into a single, concise sentence:\n\n{explanation}"
    )

    chain = prompt | llm
    result = chain.invoke({"explanation": explanation})

    print(f"Summarizer's Output:\n{result.content}")
    return {"summary": result.content}

In [35]:
# --- Graph Definition ---

workflow = StateGraph(ResearchState)

# Add the agent functions as nodes
workflow.add_node("researcher", researcher_agent)
workflow.add_node("summarizer", summarizer_agent)

# Define the edges, which control the flow
workflow.add_edge("researcher", "summarizer")
workflow.add_edge("summarizer", END)

# Set the entry point
workflow.set_entry_point("researcher")

# Compile the graph
app = workflow.compile()

In [36]:
# --- Main Execution Block ---
if __name__ == "__main__":
    topic = input("Please enter a topic for the agents to research and summarize: ")

    inputs = {"topic": topic}
    final_state = app.invoke(inputs)

    print("\n--- FINAL SUMMARY ---")
    print(final_state['summary'])

Please enter a topic for the agents to research and summarize: Cosmic Entity
---RESEARCHER (with SerpApi Web Search)---
Search Results:
['Powerful primordial beings. Exalted and enigmatic, they are often the subject of statues erected across the universe, such as at the Gates of Eternity.', 'Could anyone explain the hierarchy of the major cosmic beings in the Marvel universe? What roles do they have? How are they related to each ...', 'A cosmic entity is any being so great it can inherently affect entire planets, stars, galaxies, and possibly even the entire universe.', 'Cosmic entity can refer to: Contents. 1 Cosmology; 2 Religion and spirituality; 3 Occultism and Theosophy; 4 Popular culture. Cosmology.', 'The archetype of cosmological entities, inherently supernatural beings who have vast powers over the cosmos.', 'The Cosmic Entities are the characters who transcend the physical realm of the universe and possess immense power over other beings.', 'Marvel Cosmic entities ... List it